In [ ]:
# Setup
# For more information, visit:
# https://python.langchain.com/docs/tutorials/llm_chain/#setup

%pip install langchain python-dotenv
%pip install -qU langchain-openai
%pip install 'langserve[all]'

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
# Using Language Models
# For more information, visit:
# https://python.langchain.com/docs/tutorials/llm_chain/#using-language-models

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c97dde75-f1bc-4ee2-88aa-4b9031b95107-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [5]:
# OutputParsers
# For more information, visit:
# https://python.langchain.com/docs/tutorials/llm_chain/#outputparsers

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)


'Ciao!'

In [6]:
chain = model | parser

chain.invoke(messages)

'Ciao!'

In [7]:
# Prompt Templates
# For more information, visit:
# https://python.langchain.com/docs/tutorials/llm_chain/#prompt-templates

from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [8]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

In [9]:
# Chaining together components with LCEL
# For more information, visit:
# https://python.langchain.com/docs/tutorials/llm_chain/#chaining-together-components-with-lcel

chain = prompt_template | model | parser

chain.invoke({"language": "italian", "text": "hi"})


'ciao'

In [12]:
# Serving with LangServe - Client
# For more information, visit:
# https://python.langchain.com/docs/tutorials/llm_chain/#client
# Need to run the server first `python ./server.py`

from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "chinese", "text": "hi"})


'你好'